In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Cleaned_Data.csv')

In [35]:
df = df.sort_values(by=['distinct_id', 'time'])
df['ae_session_length'] = df['ae_session_length'].fillna('-1')
df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
df['ae_session_length'] = df['ae_session_length'].astype(float)
df

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
83729,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50.000000,-1.0,Mumbai,Maharashtra,IN,1,1,NaN,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
62403,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54.000000,-1.0,Ahmedabad,Gujarat,IN,1,1,NaN,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
84492,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84494,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35.000000,-1.0,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84495,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02.000000,30.1,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84479,Signup Success,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:06.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84442,Onboarding - Allow Location,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:17.000000,-1.0,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84441,Onboarding - Allow Notifications,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:19.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84440,Feed - Scroll Down,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:20.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84388,Search - Tap Home Bar,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:21.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0


In [39]:
df[df['name_x'] == 'AppOpen']

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
84492,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
78804,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:49:49.000000,-1.0,Los Altos,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
55831,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-04 16:28:25.000000,-1.0,Austin,Texas,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
55669,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-06 04:02:51.000000,-1.0,San Antonio,Texas,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
45112,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-09 13:10:16.000000,-1.0,San Antonio,Texas,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
45201,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-09 15:45:25.000000,-1.0,Austin,Texas,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
44585,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-11 02:13:31.000000,-1.0,Austin,Texas,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
44564,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-11 06:00:03.000000,-1.0,San Antonio,Texas,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
44392,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-11 13:15:26.000000,-1.0,Austin,Texas,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
33171,AppOpen,0025A8A4-2590-4ECA-8CE8-419D710AE46F,2019-11-21 03:25:46.000000,-1.0,Washington,District of Columbia,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0


In [46]:
single = df[df['locationSetting'].isna()]
single

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown


In [11]:
df['name_x'].unique()

array(['$ae_first_open', 'AppOpen', 'Visit Signup/Login Page',
       '$ae_session', 'Signup Success', 'Onboarding - Allow Location',
       'Onboarding - Allow Notifications', 'Feed - Scroll Down',
       'Search - Tap Home Bar', 'Search - Tap Return',
       'Search - Results Loaded', 'Search - Change Section',
       'Visit Menu Page', 'Menu Page - Accuracy Popup',
       'Menu Page - Switch Menus', 'Search - Tap Result',
       'Feed - Tap Dish', 'Nearby - Tap Dish Maps',
       'Sidebar - Go to Followers', 'Rating Flow - Start',
       'Rating Flow - Rate', 'Rating Flow - Complete',
       'Edit Rating - Exit', 'Menu Page - Filter Items',
       'Edit Rating - Edit', 'Rating Flow - Resubmit',
       'Permissions for Locations', '$ae_updated',
       'Search - Tap Nearby Restaurant', 'Signup - Skip',
       'Home - Explore Nearby', 'Home - Order Delivery',
       'Profile - Change Section', 'Search - Tap to Request',
       'Request Restaurant', 'Visit Home Page', 'Signup - Sign In